In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

Using TensorFlow backend.


In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
FOLDERNAME ='Music-Generation-Using-Deep-Learning-master/'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive
%cd $FOLDERNAME


data_directory = "Data2/"
data_file = "Data_Jazz.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = 'Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/Music-Generation-Using-Deep-Learning-master


In [0]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) 
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64): 
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  
            for i in range(0, 64):  
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1
        yield X, Y

In [0]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [0]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) 
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) 
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        

        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_Jazz_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))


In [6]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 44
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             22528     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_2 (LSTM)                (16, 64, 256)             525312    
___________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Batch: 1, Loss: 3.783466339111328, Accuracy: 0.017578125
Batch: 2, Loss: 3.7456023693084717, Accuracy: 0.2724609375
Batch: 3, Loss: 3.6448774337768555, Accuracy: 0.23828125
Batch: 4, Loss: 3.3214807510375977, Accuracy: 0.27734375
Batch: 5, Loss: 2.9808478355407715, Accuracy: 0.2724609375
Batch: 6, Loss: 2.9437856674194336, Accuracy: 0.228515625
Batch: 7, Loss: 2.778581142425537, Accuracy: 0.2353515625
Batch: 8, Loss: 2.76043701171875, Accuracy: 0.263671875
Batch: 9, Loss: 2.785581111907959, Accuracy: 0.263671875
Batch: 10, Loss: 2.7361700534820557, Accuracy: 0.2666015625
Batch: 11, Loss: 2.7369489669799805, Accuracy: 0.2724609375
Batch: 12, Loss: 2.6680068969726562, Accuracy: 0.2822265625
Batch: 13, Loss: 2.6804370880126953, Accuracy: 0.2548828125
Batch: 14, Loss: 2.6781952381134033, Accuracy: 0.255859375
Batch: 15, Loss: 2.6814606189727783, Accuracy: 0.255859375
Batch: 16, Loss: 2.6680350303649902, Accuracy: 0.2568359375
Batch: 17, Loss: 2.6846537590026855, Accuracy: 0.265625
Batch: 1